#### Imagine this.
A colleague is asking our help with fixing a script named disk_usage.py.<br>
The goal of the script is to check how much disk space is currently used, and print an error if it's too little space for normal operation.<br><br>
**But the script is currently broken because it has a few bugs.**<br>
Before we change anything, let’s make a couple copies of the script.<br>
We'll add _original to one copy, which we’ll keep unmodified and use for comparison<br>and _fixed to the other copy, which we’ll use to repair our fix.<br>

In [1]:
#firstly, let's look at the original file

%pycat ./_02/disk_usage.py

import shutil

def check_disk_usage(disk,min_absolute,min_percent):
    """returns True if there is enough free disk space, false otherwise."""
    du = shutil.disk_usage(disk)
    
    #calculate the percentage of free space
    percent_free = 100* du.free /du.total
    
    #caculate how many free gigabytes
    gigabytes_free = du.free /2**30
    if percent_free <min_percent or gigabytes_free < min_absolute:
        return False
    return True

#Check for at least 2GB and 10% free
if not check_disk_usage("/",2*2**30,10):
    print("ERROR: Not enough disk space")
    return 1
print ("Everything ok")
return 0


In [3]:
#Let's copy the original script.

In [2]:
!cp ./_02/disk_usage.py ./_02/disk_usage_original.py
!cp ./_02/disk_usage.py ./_02/disk_usage_fixed.py


In [4]:
#To test out, let's run it.
%run ./_02/disk_usage_fixed.py

SyntaxError: 'return' outside function (disk_usage_fixed.py, line 19)

#### Error[1]
As you can see, there is a return that's not inside any function.<br>
In python, we can only use return statements inside functions.<br>
How do we fix this?
1. **Turning the current code into a function** and call that function from the main part of our script.
2. Or we could use **sys.exit** to make the return number of the exit code of our script.

For now, let's just go with the second option. 

In [27]:
%pycat ./_02/disk_usage_fixed.py 

import shutil
import sys

def check_disk_usage(disk,min_absolute,min_percent):
    """returns True if there is enough free disk space, false otherwise."""
    du = shutil.disk_usage(disk)
    
    #calculate the percentage of free space
    percent_free = 100* du.free /du.total
    
    #caculate how many free gigabytes
    gigabytes_free = du.free /2**30
    if percent_free <min_percent or gigabytes_free < min_absolute:
        return False
    return True

#Check for at least 2GB and 10% free
if not check_disk_usage("/",2*2**30,10):
    print("ERROR: Not enough disk space")
    sys.exit(1)
print ("Everything ok")
sys.exit(0)


#### Let's execute the script we've just fixed


In [28]:
%run ./_02/disk_usage_fixed.py

ERROR: Not enough disk space


SystemExit: 1

#### Assuming that we know we actually have some free space, this is just yet another bug we gotta fix.
If you look closely at the code, you might notice that the script is converting to gigabytes twice. 
- once when calling the function, and once inside the function
    - gigabytes_free = du.free /2**30
    - if not check_disk_usage("/",2*2**30,10):
    
To fix this, let's just remove '\*2\*\*30' when calling the function

In [29]:
%pycat ./_02/disk_usage_fixed.py 

import shutil
import sys

def check_disk_usage(disk,min_absolute,min_percent):
    """returns True if there is enough free disk space, false otherwise."""
    du = shutil.disk_usage(disk)
    
    #calculate the percentage of free space
    percent_free = 100* du.free /du.total
    
    #caculate how many free gigabytes
    gigabytes_free = du.free /2**30
    if percent_free <min_percent or gigabytes_free < min_absolute:
        return False
    return True

#Check for at least 2GB and 10% free
if not check_disk_usage("/",2,10):
    print("ERROR: Not enough disk space")
    sys.exit(1)
print ("Everything ok")
sys.exit(0)


In [30]:
#Let's run that!
%run ./_02/disk_usage_fixed.py 

Everything ok


#### Now we can send the fixed script to your colleague so they can fix it. 
- diff -u disk_usage_original.py disk_usage_fixed.py > disk_usage.diff

In [34]:
!diff -u ./_02/disk_usage_original.py ./_02/disk_usage_fixed.py > ./_02/disk_usage.diff

In [35]:
#Let's check that out

!cat ./_02/disk_usage.diff

--- ./_02/disk_usage_original.py	2021-11-12 17:03:47.000000000 +0900
+++ ./_02/disk_usage_fixed.py	2021-11-12 16:59:46.000000000 +0900
@@ -15,8 +15,8 @@
     return True
 
 #Check for at least 2GB and 10% free
-if not check_disk_usage("/",2*2**30,10):
+if not check_disk_usage("/",2,10):
     print("ERROR: Not enough disk space")
-    return 1
+    sys.exit(1)
 print ("Everything ok")
-return 0
\ No newline at end of file
+sys.exit(0)
\ No newline at end of file


#### If your colleague is happy with your correction, they will do the following


In [36]:
!patch ./_02/disk_usage.py < ./_02/disk_usage.diff

patching file ./_02/disk_usage.py


In [37]:
#Let's check it out
%run ./_02/disk_usage.py

Everything ok


#### So we've now seen how we can look at differences between files, generate diff together to gather our changes.

But then again, this is still a lot of manual work which version control system can really help.